In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import sys
sys.path.append('../../')

### Use a Event Trigger to Make Selections: **Seems not working**

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

output_notebook()

In [ ]:
from bokeh.models import RangeSlider
from bokeh.layouts import column
from bokeh.events import SelectionGeometry
from hover.utils.bokeh_helper import servable

def almost_global_select(figure):
    select_event = SelectionGeometry(
        figure,
        geometry={
            "type": "poly",
            "x": [-1e4, -1e4, 1e4, 1e4],
            "y": [-1e4, 1e4, 1e4, -1e4],
            "sx": [None, None, None, None],
            "sy": [None, None, None, None],
        },
    )
    return select_event

@servable()
def burner():
    """
    Trying to simulate ploygon-based selections.
    """
    df = pd.DataFrame({
        'x': np.random.uniform(-1.0, 1.0, size=(100,)).tolist(),
        'y': np.random.uniform(-1.0, 1.0, size=(100,)).tolist(),
        'flag': [random.choice([True, False]) for i in range(100)],
    })
    
    source = ColumnDataSource(df)
    plot = figure(tools=['poly_select', 'lasso_select'])
    plot.circle(source=source)
    x_slider = RangeSlider(start=-1.0, end=1.0, value=(-0.5, 0.5), step=0.01)
    y_slider = RangeSlider(start=-1.0, end=1.0, value=(-0.5, 0.5), step=0.01)
    
    def slider_callback(attr, old, new):
        x_l, x_r = x_slider.value
        y_d, y_u = y_slider.value
        select_event = SelectionGeometry(
            plot,
            geometry={
                "type": "poly",
                "x": [x_l, x_l, x_r, x_r],
                "y": [y_d, y_u, y_u, y_d],
                #"sx": [None, None, None, None],
                #"sy": [None, None, None, None],
            },
        )
        plot._trigger_event(select_event)
        # use a patch to verify the polygon
        plot.patch([x_l, x_l, x_r, x_r], [y_d, y_u, y_u, y_d], alpha=0.2, line_width=1)
        # check the number of selected points
        print(len(source.selected.indices), end="\r")
    
    x_slider.on_change('value', slider_callback)
    y_slider.on_change('value', slider_callback)
    
    return column(x_slider, y_slider, plot)

In [ ]:
handle = burner()
show(handle)